# Teoria de Numeros

Supongamos que se requiere trasmitir un mensaje codificado usando una
función $f : D \mapsto D$, donde $D = \{A, B, C , . . . , Z\}$. Dado que el alfabeto
contiene 26 letras, podemos usar el siguiente metodo:

\begin{align}
f(p) = (p + k) \operatorname{mod} 26
\end{align}

Donde $p$ es la poisición del carácter que vamos a reemplazar y $0 < k < 25$

In [1]:
import string


def encrypt(msg,private_key):
    enc_msg=''
    SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'
    code={v:k for k,v in enumerate(SYMBOLS)}
    reverse_code={k:v for k,v in enumerate(SYMBOLS)}
    for i in range(len(msg)):
        enc_msg+=reverse_code[(code[msg[i]]+private_key)%len(SYMBOLS)]
    return enc_msg

In [2]:
encrypt('CAESAR CIPHER DEMO',4)

'GEIWEVAGMTLIVAHIQS'

In [3]:
from secrets import token_bytes
from typing import Tuple

def random_key(length):
    tb = token_bytes(length)
    return int.from_bytes(tb, 'big')


In [4]:
def encrypt_one_time(msg):
    original_bytes = msg.encode()
    dummy = random_key(len(original_bytes))
    original_key = int.from_bytes(original_bytes, "big")
    encrypted = original_key ^ dummy # XOR
    return dummy, encrypted

In [5]:
(key1,key2)=encrypt_one_time('CAESAR CIPHER DEMO')

In [6]:
key1,key2

(12260286572352991032056079176271561902328264,
 18118328172115355222363471057789896599553159)

In [7]:
def decrypt_one_time(key1, key2):
    decrypted = key1 ^ key2 # XOR
    temp= decrypted.to_bytes((decrypted.bit_length()+ 7) // 8, "big")
    return temp.decode()

In [8]:
decrypt_one_time(key1,key2)

'CAESAR CIPHER DEMO'

# Algoritmo RSA

El cifrado RSA se basa en la factorizacion de numeros enteros. A
diferencia del cifrado Cesar, RSA utiliza una llave publica $(e, n)$ y una privada $(d, n)$. 

Para codificar el mensaje m usamos la llave publica $e$ del receptor.

\begin{align}
c \equiv m^e  \operatorname{mod} n
\end{align}

Para decodificar el mensaje, usamos la llave privada $n$ del emisor.

\begin{align}
m \equiv c^d  \operatorname{mod} n
\end{align}

Los pasos para crear las llaves son:

* Crear de manera aleatoria dos enteros primos de gran tamano : $p$ y $q$.
* Multiplicar los enteros de manera de generar un numero compuesto $n$.
* Crear de manera aleatoria un entero $e$, que sea relativamente primo con $(p–1) \times (q–1)$. Es decir que el maximo divisor comun $\operatorname{GCD}(e,(p–1) \times (q–1))==1$
* Calcular el inverso modular de $e$, el cual llamaremos $d$. 

In [13]:
import math 

def isPrime(n):
    if n<2:
        return False
    for i in range(2,int(math.sqrt(n)+1)):
        if n % i == 0:
            return False
    return True

print('9 es primo : ',isPrime(9))
print('11 es primo : ',isPrime(11))

9 es primo :  False
11 es primo :  True
19 es primo :  True


In [11]:
getPrimes = lambda n : [n for n in range(n) if isPrime(n)]

getPrimes(20)

[2, 3, 5, 7, 11, 13, 17, 19]

In [30]:
import random 

def generateLargePrime(keysize=1024):
    num=0
    while not isPrime(num):
        num = random.randrange(2**(keysize-1), 2**(keysize))
    return num

In [31]:
def generate_composite(keysize):
    # Creates public/private keys keySize bits in size.
    p = 0
    q = 0
    print('Generating p,q primes...')
    while p == q:
        p = generateLargePrime(keysize)
        q = generateLargePrime(keysize)
        n = p * q
    print('done!')
    return n,p,q

In [169]:
# Euclidean Algorithm
def gcd(x, y): 
    while(y): 
        x, y = y, x % y 
    return x 

# extended Euclidean Algorithm
def egcd(a, b):
    """return (g, x, y) such that a*x + b*y = g = gcd(a, b)"""
    if a == 0:
        return (b, 0, 1)
    else:
        b_div_a, b_mod_a = divmod(b, a)
        g, x, y = egcd(b_mod_a, a)
        return (g, y - b_div_a * x, x)

In [170]:
egcd(37, 60)

(1, 13, -8)

In [171]:
# x=2^4*3^5*7^4*11^1
a=102685968
# y=2^3*3^7*7^2*11^2
b=103733784
z=gcd(a,b)

In [158]:
z==2**3*3**5*7**2*11**1 

True

In [159]:
g,x,y=egcd(a,b)

In [160]:
g==x*a+y*b

True

In [184]:
def generate_rsa(keysize):
    n,p,q=generate_composite(keysize)
    while True:
        e = random.randrange(2**(keysize-1), 2**(keysize))
        if gcd(e,(p-1)*(q-1))==1:
            break
    print('p:',p)
    print('q:',q)
    print('e:',e)    
    g,d,y=egcd(e,(p-1)*(q-1))
    print('d:',d) 
    return (e,n),(d,n)

In [185]:
public_key,private_key=generate_rsa(20)

Generating p,q primes...
done!
p: 737729
q: 1040203
e: 920105
d: 323244538649


In [186]:
public_key

(920105, 767387918987)

In [187]:
private_key

(323244538649, 767387918987)

In [188]:
public_key[1]//(public_key[0]*private_key[0])

0

In [189]:
gcd(public_key[0],private_key[0])

1

In [115]:
SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'

len(SYMBOLS)**169<2**1024

True